#### 1. 使用LangChain的 TextLoader

In [26]:
from langchain_community.document_loaders import TextLoader
import pprint

text_loader = TextLoader("../../90-文档_Data/test/人物角色.json")
text_documents = text_loader.load()

# 打印每个文档的元数据
for doc in text_documents:
    pprint.pprint(doc.model_dump())

{'id': None,
 'metadata': {'source': '../../90-文档_Data/test/人物角色.json'},
 'page_content': '{\n'
                 '    "gameTitle": "西游记",\n'
                 '    "basicInfo": { "engine": "虚幻引擎5", "releaseDate": '
                 '"2024-08-20", "genre": "动作角色扮演", "platforms": ["PC", "PS5", '
                 '"Xbox Series X/S"], "supportedLanguages": ["简体中文", "繁体中文", '
                 '"英语", "日语", "韩语"] },\n'
                 '    "mainCharacter": { "name": "孙悟空", "backstory": '
                 '"混沌初开之时，盘古开天辟地，天地灵气凝结成仙石，其中孕育出一只石猴。这只石猴拜师菩提老祖，修得一身本领，后被赐名孙悟空。", '
                 '"abilities": ["七十二变", "筋斗云", "火眼金睛", "金刚不坏之身"], "weapons": '
                 '["如意金箍棒", "法器", "神通符咒"], "combatStyle": '
                 '"基于变身系统的高速格斗，结合中国传统神话元素的技能组合" },\n'
                 '    "supportCharacters": [\n'
                 '      { "name": "白龙马", "identity": "八部天龙之一", "background": '
                 '"原为西海龙王三太子敖烈，因冒犯天条被贬为马，后随唐僧西行取经，成为孙悟空的伙伴", "abilities": '
                 '["水遁", "腾云驾雾", "

#### 2. 使用LangChain的JSONLoader

In [27]:
from langchain_community.document_loaders import JSONLoader
import pprint

print("1. 主角信息：")
main_loader = JSONLoader(
    file_path="../../90-文档_Data/test/人物角色.json",
    jq_schema='.mainCharacter | "姓名：" + .name + "，背景：" + .backstory',
    text_content=True
)
main_char = main_loader.load()
# 打印每个文档的元数据
for doc in main_char:
    pprint.pprint(doc.model_dump())


print("\n2. 支持角色信息：")
support_loader = JSONLoader(
    file_path="../../90-文档_Data/test/人物角色.json",
    jq_schema='.supportCharacters[] | "姓名：" + .name + "，背景：" + .background',
    text_content=True
)
support_chars = support_loader.load()
# 打印每个文档的元数据
for doc in support_chars:
    pprint.pprint(doc.model_dump())

1. 主角信息：
{'id': None,
 'metadata': {'seq_num': 1,
              'source': '/home/ylins/project/rag-related/2.RAG核心技术拆解/90-文档_Data/test/人物角色.json'},
 'page_content': '姓名：孙悟空，背景：混沌初开之时，盘古开天辟地，天地灵气凝结成仙石，其中孕育出一只石猴。这只石猴拜师菩提老祖，修得一身本领，后被赐名孙悟空。',
 'type': 'Document'}

2. 支持角色信息：
{'id': None,
 'metadata': {'seq_num': 1,
              'source': '/home/ylins/project/rag-related/2.RAG核心技术拆解/90-文档_Data/test/人物角色.json'},
 'page_content': '姓名：白龙马，背景：原为西海龙王三太子敖烈，因冒犯天条被贬为马，后随唐僧西行取经，成为孙悟空的伙伴',
 'type': 'Document'}
{'id': None,
 'metadata': {'seq_num': 2,
              'source': '/home/ylins/project/rag-related/2.RAG核心技术拆解/90-文档_Data/test/人物角色.json'},
 'page_content': '姓名：红孩儿，背景：牛魔王与铁扇公主之子，修炼火焰三昧，掌握三昧真火',
 'type': 'Document'}
{'id': None,
 'metadata': {'seq_num': 3,
              'source': '/home/ylins/project/rag-related/2.RAG核心技术拆解/90-文档_Data/test/人物角色.json'},
 'page_content': '姓名：六耳猕猴，背景：天地间与美猴王最像的存在，有着与孙悟空相似的能力',
 'type': 'Document'}


#### 3. 使用LangChain的加载网页

LangChain中各种网页加载器
| 文档加载器 | 说明 | Package/API | 特点 |
|------------|------|-------------|-------|
| WebBaseLoader | 使用 urllib 和 BeautifulSoup 加载并解析HTML网页 | Package | 简单易用，适合基本网页内容抓取 |
| UnstructuredLoader | 使用 Unstructured 解析和加载网页 | Package | 支持多种复杂网页结构，适合异构内容处理 |
| RecursiveURLLoader | 从根URL递归抓取所有子链接 | Package | 自动化链接抓取，适合大规模站点抓取 |
| SitemapLoader | 从给定的站点地图抓取所有页面 | Package | 高效解析站点结构，快速获取所有页面内容 |
| FirecrawlLoader | 可本地部署的API服务，托管版本有免费额度 | API | 灵活可扩展，适合需要实时抓取和转换的应用 |

In [34]:
# 使用WebBaseLoader加载网页
import bs4
from langchain_community.document_loaders import WebBaseLoader
page_url = "https://zh.wikipedia.org/wiki/黑神话：悟空"

# loader = WebBaseLoader(web_paths=[page_url])
# docs = []
# docs = loader.load()
# assert len(docs) == 1
# doc = docs[0]
# print(f"{doc.metadata}\n")
# print(doc.page_content.strip()[:3000])

# 只解析文章的主体部分
loader = WebBaseLoader(
    web_paths=[page_url],
    bs_kwargs={
        # 指定只解析id为bodyContent的标签内容
        "parse_only": bs4.SoupStrainer(id="bodyContent"),
    },
    # bs_get_text_kwargs={
    #     "separator": " | ",  # 设置文本分隔符为 " | "
    #     "strip": True,       # 去除文本前后的空白字符
    # },
)
docs = []
docs = loader.load()
assert len(docs) == 1
doc = docs[0]
print(f"{doc.metadata}\n")
print(doc.page_content)

{'source': 'https://zh.wikipedia.org/wiki/黑神话：悟空'}






维基百科，自由的百科全书


此條目的引用需要清理，使其符合格式。 (2025年1月19日)参考文献应符合正确的引用、脚注及外部链接格式。

黑神话：悟空类型動作角色扮演[1]平台Microsoft WindowsPlayStation 5Xbox Series X/S（暂未发售）开发商游戏科学发行商全球：游戏科学臺灣：4Divinity制作人冯骥、杨奇编剧冯骥美术杨奇[2]音乐8082 Audio引擎虚幻引擎5[3]模式单人游戏发行日
2024年8月20日
Microsoft Windows全球：2024年8月20日PlayStation 5全球：2024年8月20日臺灣：2024年11月1日全球：2024年12月（實體豪華版）Xbox Series X/S全球：待定

《黑神话：悟空》是一款由游戏科学开发和发行的动作角色扮演游戏，被新华社称为中国首款“3A游戏”[4]，游戏于2024年8月20日登入Windows（Steam、Epic Games、WeGame）及PlayStation 5平台，Xbox Series X/S版的发售日期待定。
游戏内容改编自中国四大名著之一的《西游记》[5]，在正式发布前，游戏已获得业界媒体与评论家们的普遍好评，称赞其在战斗系统、视觉设计以及世界观方面的构建。游戏上线后迅速登顶多个平台的销量榜首，官方宣布首发3天内全球销量超过1000万份[6]，发售后一个月内全球销量超过2000万份[7]，成为有史以来销售速度最快的游戏之一。


玩法[编辑]
《黑神话：悟空》是一款动作角色扮演游戏，[8]采用单人模式和第三人称视角。尽管在游戏发布前因為本作借鑒了不少魂类遊戲要素，曾有人讨论其是否属于魂类游戏，但其开发商游戏科学并不认为它属于这一类型[9]，许多评论者也认为该游戏并不完全符合“魂类”游戏的定义。[10][11]
游戏的设计灵感源于中国四大名著之一——古典神魔小說《西游记》，玩家将操控一位名为“天命人”的花果山灵明石猴。主角的主要武器是如意金箍棒等棍棒类武器，能在战斗中伸缩自如。游戏的战斗系统包括三种主要棍法：劈棍、戳棍和立棍，为战斗提供多样化选择。战斗还涉及资源管理，玩家可以积累气力值，用于连招或蓄力重击。冲刺、闪避